In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

### 1
Для выполнения задания вы можете самостоятельно создать Python-ноутбук, а в LMS прикладывать лишь требуемые результаты.

Задача: Прогнозирование вероятности покупки товара в интернет-магазине

Студент B посетил интернет-магазин и просмотрел товары из категории "Электроника", сумма его предыдущей покупки составила 1500 рублей, а длительность сессии - 20 минут.

У нас есть обученная модель логистической регрессии с коэффициентами:

сумма покупки - 0.002

категория "Электроника" - 0.5

длительность сессии - 0.1

свободный коэффициент - -5

Получите вероятность покупки студентом B в интернет-магазине и укажите ответ, округлив его до второго знака после точки.

In [2]:
1/(1+np.exp(-(1500*0.002+1*0.5+0.1*20+(-5))))

0.6224593312018546

### 2
Имеется датасет для предсказания оценок студентов. Но мы возьмём немного измененную версию данных, скачать датасет.

Сделайте разбиение на обучение и тест с соотношением 80/20 и random_state=10.

Целевой признак grade: 0 - оценка ниже среднего, 1 - оценка выше среднего.

Обучите логистическую регрессию из sklearn на базовых параметрах для решения поставленной задачи и укажите вес, который получился после обучения модели, при признаке studytime, округлив его до второго знака после точки.

In [3]:
df_1 = pd.read_csv("student.csv")
df_1

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,grade
0,18,4,4,2,2,0,4,3,4,1,1,3,4,1
1,17,1,1,1,2,0,5,3,3,1,1,3,2,1
2,15,1,1,1,2,0,4,3,2,2,3,3,6,1
3,15,4,2,1,3,0,3,2,2,1,1,5,0,2
4,16,3,3,1,2,0,4,3,2,1,2,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,19,2,3,1,3,1,5,4,2,1,2,5,4,1
645,18,3,1,1,2,0,4,3,4,1,1,1,4,2
646,18,1,1,2,2,0,1,1,1,1,1,5,6,1
647,17,3,1,2,1,0,2,4,5,3,4,2,6,1


In [14]:
X = df_1.drop(["grade"], axis=1)
y = df_1["grade"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10)
model_1 = LogisticRegression()
model_1.fit(X_train, y_train)
round(model_1.coef_[0,4], 2)

0.38231757041723974

### 3
Продолжайте работать с данными с предыдущего задания.

Вероятней всего (если у вас не отключены предупреждения) Python выдал 

ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT., 

такое встречается, если в данных есть признаки разных масштабов. Давайте исправим это с помощью масштабирования данных через стандартизацию.

Обучите модель StandardScaler на тренировочной выборке.

Преобразуйте тренировочные и тестовые данных с помощью стандартизации

Укажите, какие диапазоны получились у признака age на тестовой выборке - значение min и max. 

In [16]:
scaler_1 = StandardScaler()
scaler_1.fit(X_train)
X_train_scaled = scaler_1.transform(X_train)
X_test_scaled = scaler_1.transform(X_test)

ans_sorted = np.sort(X_test_scaled[:, 0])
print(f"min: {round(ans_sorted[0],2)}, max: {round(ans_sorted[-1],2)}")

min: -1.47, max: 4.4


### 4
Продолжайте работать с масштабированными данными с предыдущего задания.

Обучите модель логистической регрессии с базовыми параметрами на стандартизованных данных и посчитайте метрику accuracy на тренировочной и тестовой выборках и укажите их, округлив до второго знака после точки.

In [6]:
model_1.fit(X_train_scaled, y_train)
y_pred_test = model_1.predict(X_test_scaled)
y_pred_train = model_1.predict(X_train_scaled)
print(f"test: {round(accuracy_score(y_test, y_pred_test), 2)} \ntrain: {round(accuracy_score(y_train, y_pred_train), 2)}")

test: 0.78 
train: 0.76


### 5
Продолжайте работать с данными с предыдущего задания.

Возьмите НЕ масштабированные данные, т.к. сейчас будем их усложнять для более качественной модели.

Для удобства можете пользоваться pipeline из sklearn, в котором будут:

Модель полиномизации признаков до 4 степени PolynomialFeatures

Модель масштабирования данных StandardScaler

Логистическая регрессия БЕЗ регуляризации

По умолчанию в логистической регрессии в sklearn есть L2 регуляризация, чтобы её отключить, нужно поменять параметр penalty.

Обучите пайплайн на тренировочной выборке и посчитайте метрику accuracy на тренировочной и тестовой выборках и укажите их, округлив до второго знака после точки.

In [7]:
pipe = Pipeline([('preprocessor', PolynomialFeatures(4)),('scaler', StandardScaler()), ('logistic', LogisticRegression(penalty=None))])

pipe.fit(X_train, y_train)
y_pred_train = pipe.predict(X_train)
y_pred_test = pipe.predict(X_test)

print(f"test: {round(accuracy_score(y_test, y_pred_test), 2)} \ntrain: {round(accuracy_score(y_train, y_pred_train), 2)}")

test: 0.68 
train: 0.95


### 6
Продолжайте работать с данными с предыдущего задания.

Снова вероятней всего (если у вас не отключены предупреждения) Python выдал ConvergenceWarning: lbfgs failed to converge (status=1): STOP: TOTAL NO. of ITERATIONS REACHED LIMIT., хотя масштабирование данных сделано, так же эту ошибку можно избежать, если увеличить количество итераций в логистической регрессии, увеличьте параметр max_iter до 600.

Так же метрика на тесте стала меньше, потому что модель переобучается, нужно и это исправить с помощью регуляризации.

Для удобства можете пользоваться pipeline из sklearn, в котором будут:

Модель полиномизации признаков до 4 степени PolynomialFeatures

Модель масштабирования данных StandardScaler

Логистическая регрессия с L2 регуляризацией

Сделайте перебор параметра C в логистической регрессии, который из себя представляет 1/коэффициент регуляризации. Значения C перебирайте из следующего списка: [0.0001, 0.001, 0.01, 0.1, 1].

Укажите, самую лучшую метрику на тесте, округлив её до второго знака после точки.

In [17]:
answer = []

for coef in [0.0001, 0.001, 0.01, 0.1, 1]:
    pipe = Pipeline([('preprocessor', PolynomialFeatures(4)),('scaler', StandardScaler()), ('logistic', LogisticRegression(C=coef, max_iter=600))])
    
    pipe.fit(X_train, y_train)
    y_pred_test = pipe.predict(X_test)

    answer.append(round(accuracy_score(y_test, y_pred_test), 2))
max(answer)

0.81

### 7
Имеется датасет для чаевых для таксиста . Но мы возьмём немного измененную версию данных. Скачать датасет.

Сделайте разбиение на обучение и тест с соотношением 80/20 и random_state=10.

Целевой признак tip_class.

Обучите логистическую регрессию из sklearn с параметром max_iter=400 для решения поставленной задачи.

Изучите масштаб в данных и если это требуется примените StandardScaler.

Укажите метрики accuracy на тренировочной и тестовой выборках, округлив их до второго знака после точки. 

In [9]:
df_2 = pd.read_csv("taxi.csv")
df_2

,trip_duration,distance_traveled,num_of_passengers,fare,miscellaneous_fees,total_fare,surge_applied,tip_class
0,748.0,2.75,1.0,75.00,6.300,105.300,0,over med
1,1187.0,3.43,1.0,105.00,13.200,142.200,0,over med
2,329.0,2.09,1.0,45.00,13.200,70.200,0,under med
3,355.0,1.74,1.0,45.00,26.625,71.625,1,low
4,1288.0,5.21,1.0,116.25,26.325,178.575,1,high
...,...,...,...,...,...,...,...,...
137160,729.0,2.49,1.0,75.00,6.200,97.200,0,under med
137161,1617.0,8.42,1.0,150.00,5.800,202.800,0,high
137162,438.0,1.29,1.0,48.75,34.575,95.325,1,under med
137163,571.0,2.82,1.0,63.75,6.000,69.750,0,low


In [10]:
X = df_2.drop(["tip_class"], axis=1)
y = df_2["tip_class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=10)

pipe = Pipeline([("scaler", StandardScaler()), ("logreg", LogisticRegression(max_iter=400))])

pipe.fit(X_train, y_train)
y_pred_train = pipe.predict(X_train)
y_pred_test = pipe.predict(X_test)

print(f"test: {round(accuracy_score(y_test, y_pred_test), 2)} \ntrain: {round(accuracy_score(y_train, y_pred_train), 2)}")

test: 0.96 
train: 0.96


### 8
Продолжайте работать с датасетом предыдущего задания.

Посчитайте, сколько получилось ошибочных предсказаний в процентном соотношении по каждому классу на тестовой выборке и укажите в ответе, округлив проценты до второго знака после точки.

Пример ответа:

low: 5.10% ошибочных предсказаний under med: 15.26% ошибочных предсказаний over med: 18.93% ошибочных предсказаний high: 6.24% ошибочных предсказаний

In [11]:
counter = {"low": 0, "under med": 0, "over med": 0, "high": 0}
errors = {"low": 0, "under med": 0, "over med": 0, "high": 0}

for pred, test in zip(y_pred_test, y_test):
    counter[pred] += 1
    if pred != test:
        errors[pred] += 1

for key in counter.keys():
    print(f"{key}: {round((errors[key]/counter[key])*100,4)} % ошибочных предсказаний")

low: 0.0303 % ошибочных предсказаний
under med: 11.3378 % ошибочных предсказаний
over med: 2.542 % ошибочных предсказаний
high: 0.3873 % ошибочных предсказаний
